# Check computation of relative SST

## Imports

In [ ]:
import xarray as xr
import pathlib
import os
import matplotlib.pyplot as plt
import seaborn as sns

sns.set(rc={"axes.facecolor": "white", "axes.grid": False})

## Load data

In [ ]:
## open data
sst = xr.open_dataset(
    list(pathlib.Path(os.environ["DATA_FP"], "mpi", "ts").glob("*.nc"))[0],
)

In [ ]:
## get bounds for defining sst
bnds = [5, 10, 15, 20, 25, 30]

## compute tropical pacific sst
for bnd in bnds:
    sst[f"sst_trop_{bnd:02d}"] = (
        sst["sst"].sel(latitude=slice(-bnd, bnd)).mean(["latitude", "longitude"])
    )

## drop spatial info
sst = sst.drop_vars(["sst", "time_bnds", "bounds_longitude", "bounds_latitude"])

## compute annual mean
sst_am = sst.groupby("time.year").mean()

## check corr. when including seasonal cycle

In [ ]:
for bnd in bnds:
    corr = xr.corr(sst[f"sst_trop_{bnd:02d}"], sst["sst_trop"]).values.item()
    print(f"{bnd:02d}: {corr:.3f}")

print()
print(f"glob: {xr.corr( sst[f"ts_glob"], sst["sst_trop"]).values.item():.3f}")

## Check for annual mean

In [ ]:
for bnd in bnds:
    corr = xr.corr(sst_am[f"sst_trop_{bnd:02d}"], sst_am["sst_trop"]).values.item()
    print(f"{bnd:02d}: {corr:.3f}")
print()
print(f"glob: {xr.corr( sst_am[f"ts_glob"], sst_am["sst_trop"]).values.item():.3f}")

Plot

In [ ]:
sel = lambda x: x
fig, ax = plt.subplots(figsize=(8, 3))

ax.plot(sst_am.year, sel(sst_am["sst_trop"]))
ax.plot(sst_am.year, sel(sst_am["sst_trop_30"]))
ax.plot(sst_am.year, 10.6 + sel(sst_am["ts_glob"]))

plt.show()

In [ ]:
sel = lambda x: x.sel(time=x.time.dt.month == 9)
fig, ax = plt.subplots(figsize=(8, 3))

ax.plot(sel(sst["sst_trop"]).time, sel(sst["sst_trop"]))
ax.plot(sel(sst["sst_trop_30"]).time, sel(sst["sst_trop_30"]))

plt.show()